# Predict Future Sales 실전 문제

#### !_contents_!
- 훈련 데이터에서 테스트 데이터에 있는 상점ID만 추출
- 추가적인 피처 엔지니어링을 통한 성능 개선 시도

# 1. Loading data, libraries (+data downcasting)

In [1]:
import numpy as np
import pandas as pd

from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

import lightgbm as lgb

import warnings
warnings.filterwarnings(action='ignore')

# 데이터 경로 지정
data_path = '/kaggle/input/competitive-data-science-predict-future-sales/'

sales_train = pd.read_csv(data_path + 'sales_train.csv')
shops = pd.read_csv(data_path + 'shops.csv')
items = pd.read_csv(data_path + 'items.csv')
item_categories = pd.read_csv(data_path + 'item_categories.csv')
test = pd.read_csv(data_path + 'test.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

In [2]:
sales_train = sales_train.rename(columns={'date': '날짜', 
                                          'date_block_num': '월ID',
                                          'shop_id': '상점ID',
                                          'item_id': '상품ID',
                                          'item_price': '판매가',
                                          'item_cnt_day': '판매량'})

shops = shops.rename(columns={'shop_name': '상점명',
                              'shop_id': '상점ID'})

items = items.rename(columns={'item_name': '상품명',
                              'item_id': '상품ID',
                              'item_category_id': '상품분류ID'})

item_categories = item_categories.rename(columns=
                                         {'item_category_name': '상품분류명',
                                          'item_category_id': '상품분류ID'})

test = test.rename(columns={'shop_id': '상점ID',
                            'item_id': '상품ID'})

In [3]:
def downcast(df, verbose=True):
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        dtype_name = df[col].dtype.name
        if dtype_name == 'object':
            pass
        elif dtype_name == 'bool':
            df[col] = df[col].astype('int8')
        elif dtype_name.startswith('int') or (df[col].round() == df[col]).all():
            df[col] = pd.to_numeric(df[col], downcast='integer')
        else:
            df[col] = pd.to_numeric(df[col], downcast='float')
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print('{:.1f}% 압축됨'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

all_df = [sales_train, shops, items, item_categories, test]
for df in all_df:
    df = downcast(df)

54.2% 압축됨
38.6% 압축됨
54.2% 압축됨
39.9% 압축됨
70.8% 압축됨


# 2. preprocessing

In [4]:
# sales price 0~50000
sales_train = sales_train[sales_train['판매가'] > 0]
sales_train = sales_train[sales_train['판매가'] < 50000]
# sales quantity 0~1000
sales_train = sales_train[sales_train['판매량'] > 0]
sales_train = sales_train[sales_train['판매량'] < 1000]

In [5]:
# considering some shops are same each other
print(shops['상점명'][0], '||', shops['상점명'][57])
print(shops['상점명'][1], '||', shops['상점명'][58])
print(shops['상점명'][10], '||', shops['상점명'][11])
print(shops['상점명'][39], '||', shops['상점명'][40])

# in sales_train
sales_train.loc[sales_train['상점ID'] == 0, '상점ID'] = 57
sales_train.loc[sales_train['상점ID'] == 1, '상점ID'] = 58
sales_train.loc[sales_train['상점ID'] == 10, '상점ID'] = 11
sales_train.loc[sales_train['상점ID'] == 39, '상점ID'] = 40

# in test
test.loc[test['상점ID'] == 0, '상점ID'] = 57
test.loc[test['상점ID'] == 1, '상점ID'] = 58
test.loc[test['상점ID'] == 10, '상점ID'] = 11
test.loc[test['상점ID'] == 39, '상점ID'] = 40

!Якутск Орджоникидзе, 56 фран || Якутск Орджоникидзе, 56
!Якутск ТЦ "Центральный" фран || Якутск ТЦ "Центральный"
Жуковский ул. Чкалова 39м? || Жуковский ул. Чкалова 39м²
РостовНаДону ТРК "Мегацентр Горизонт" || РостовНаДону ТРК "Мегацентр Горизонт" Островной


## 🔍 Selecting Store IDs Present in the Test Dataset(실전문제 실습)

In [6]:
# 테스트 데이터에 있는 상점ID만 추출
unique_test_shop_id = test['상점ID'].unique()
sales_train = sales_train[sales_train['상점ID'].isin(unique_test_shop_id)]

In [7]:
shops['도시'] = shops['상점명'].apply(lambda x: x.split()[0])

In [8]:
shops['도시'].unique()

array(['!Якутск', 'Адыгея', 'Балашиха', 'Волжский', 'Вологда', 'Воронеж',
       'Выездная', 'Жуковский', 'Интернет-магазин', 'Казань', 'Калуга',
       'Коломна', 'Красноярск', 'Курск', 'Москва', 'Мытищи', 'Н.Новгород',
       'Новосибирск', 'Омск', 'РостовНаДону', 'СПб', 'Самара', 'Сергиев',
       'Сургут', 'Томск', 'Тюмень', 'Уфа', 'Химки', 'Цифровой', 'Чехов',
       'Якутск', 'Ярославль'], dtype=object)

In [9]:
shops.loc[shops['도시'] =='!Якутск', '도시'] = 'Якутск'

In [10]:
from sklearn.preprocessing import LabelEncoder

# 레이블 인코더 생성
label_encoder = LabelEncoder()
# 도시 피처 레이블 인코딩
shops['도시'] = label_encoder.fit_transform(shops['도시'])

## 🔍 Removing `_name` Features

In [11]:
shops = shops.drop(['상점명'], axis=1)
shops.head()

,상점ID,도시
0,0,29
1,1,29
2,2,0
3,3,1
4,4,2


In [12]:
items = items.drop(['상품명'], axis=1)
items.head()

,상품ID,상품분류ID
0,0,40
1,1,76
2,2,40
3,3,40
4,4,40


# 3. Feature Engineering: textbook ver.

## 🔍 Creating Feature: `첫 판매월`, `대분류` 

In [13]:
# 상품이 맨 처음 팔린 날을 피처로 추가
items['첫 판매월'] = sales_train.groupby('상품ID').agg({'월ID': 'min'})['월ID']
items.head()

,상품ID,상품분류ID,첫 판매월
0,0,40,NaN
1,1,76,15.0
2,2,40,NaN
3,3,40,NaN
4,4,40,NaN


In [14]:
items[items['첫 판매월'].isna()]

,상품ID,상품분류ID,첫 판매월
0,0,40,NaN
2,2,40,NaN
3,3,40,NaN
4,4,40,NaN
5,5,40,NaN
...,...,...,...
21975,21975,61,NaN
22022,22022,40,NaN
22028,22028,40,NaN
22035,22035,40,NaN


In [15]:
# 첫 판매월 피처의 결측값을 34로 대체
items['첫 판매월'] = items['첫 판매월'].fillna(34)

In [16]:
# 상품분류명의 첫 단어를 대분류로 추출
item_categories['대분류'] = item_categories['상품분류명'].apply(lambda x: x.split()[0])  

In [17]:
item_categories['대분류'].value_counts()

대분류
Игры          14
Книги         13
Подарки       12
Игровые        8
Аксессуары     7
Музыка         6
Программы      6
Карты          5
Кино           5
Служебные      2
Чистые         2
PC             1
Билеты         1
Доставка       1
Элементы       1
Name: count, dtype: int64

In [18]:
def make_etc(x):
    if len(item_categories[item_categories['대분류']==x]) >= 5:
        return x
    else:
        return 'etc'

# 5개 미만이면 'etc'로 통합
item_categories['대분류'] = item_categories['대분류'].apply(make_etc)
item_categories.head()

,상품분류명,상품분류ID,대분류
0,PC - Гарнитуры/Наушники,0,etc
1,Аксессуары - PS2,1,Аксессуары
2,Аксессуары - PS3,2,Аксессуары
3,Аксессуары - PS4,3,Аксессуары
4,Аксессуары - PSP,4,Аксессуары


In [19]:
label_encoder = LabelEncoder()
item_categories['대분류'] = label_encoder.fit_transform(item_categories['대분류'])

item_categories = item_categories.drop('상품분류명', axis=1)

## 🔍 Generating Features by Combining `월ID`, `상점ID`, `상품ID`

In [20]:
from itertools import product

train = []
# 월ID, 상점ID, 상품ID 조합 생성
for i in sales_train['월ID'].unique():
    all_shop = sales_train.loc[sales_train['월ID']==i, '상점ID'].unique()
    all_item = sales_train.loc[sales_train['월ID']==i, '상품ID'].unique()
    train.append(np.array(list(product([i], all_shop, all_item))))

idx_features = ['월ID', '상점ID', '상품ID'] # 기준 피처
train = pd.DataFrame(np.vstack(train), columns=idx_features)

In [21]:
group = sales_train.groupby(idx_features).agg({'판매량': 'sum',
                                               '판매가': 'mean'})
group = group.reset_index()
group = group.rename(columns={'판매량': '월간 판매량', '판매가': '평균 판매가'})

train = train.merge(group, on=idx_features, how='left')

train.head()

,월ID,상점ID,상품ID,월간 판매량,평균 판매가
0,0,59,22154,1.0,999.0
1,0,59,2552,NaN,NaN
2,0,59,2554,NaN,NaN
3,0,59,2555,NaN,NaN
4,0,59,2564,NaN,NaN


In [22]:
import gc

del group
gc.collect();

In [23]:
# 상품 "판매"건수 피처 추가
group = sales_train.groupby(idx_features).agg({'판매량': 'count'})
group = group.reset_index()
group = group.rename(columns={'판매량': '판매건수'})

train = train.merge(group, on=idx_features, how='left')

del group, sales_train
gc.collect()

train.head()

,월ID,상점ID,상품ID,월간 판매량,평균 판매가,판매건수
0,0,59,22154,1.0,999.0,1.0
1,0,59,2552,NaN,NaN,NaN
2,0,59,2554,NaN,NaN,NaN
3,0,59,2555,NaN,NaN,NaN
4,0,59,2564,NaN,NaN,NaN


In [24]:
# 테스트 데이터 월ID를 34로 설정
test['월ID'] = 34

# train과 test 이어붙이기
all_data = pd.concat([train, test.drop('ID', axis=1)],
                     ignore_index=True,
                     keys=idx_features)

# 결측값을 0으로 대체
all_data = all_data.fillna(0)
all_data.head()

,월ID,상점ID,상품ID,월간 판매량,평균 판매가,판매건수
0,0,59,22154,1.0,999.0,1.0
1,0,59,2552,0.0,0.0,0.0
2,0,59,2554,0.0,0.0,0.0
3,0,59,2555,0.0,0.0,0.0
4,0,59,2564,0.0,0.0,0.0


In [25]:
# 나머지 데이터 병합
all_data = all_data.merge(shops, on='상점ID', how='left')
all_data = all_data.merge(items, on='상품ID', how='left')
all_data = all_data.merge(item_categories, on='상품분류ID', how='left')

# 다운캐스팅
all_data = downcast(all_data)

del shops, items, item_categories
gc.collect();

68.9% 압축됨


In [26]:
def add_mean_features(df, mean_features, idx_features):
    # 기준 피처 확인
    assert (idx_features[0] == '월ID') and \
           len(idx_features) in [2, 3]
    
    # 파생 피처명 설정 
    if len(idx_features) == 2:
        feature_name = idx_features[1] + '별 평균 판매량'
    else:
        feature_name = idx_features[1] + ' ' + idx_features[2] + '별 평균 판매량'
    
    # 기준 피처를 토대로 그룹화해 월간 평균 판매량 구하기
    group = df.groupby(idx_features).agg({'월간 판매량': 'mean'})
    group = group.reset_index()
    group = group.rename(columns={'월간 판매량': feature_name})
    
    # df와 group 병합 
    df = df.merge(group, on=idx_features, how='left')
    # 다운캐스팅 
    df = downcast(df, verbose=False)
    # 새로 만든 feature_name 피처명을 mean_features 리스트에 추가 
    mean_features.append(feature_name)

    # garbage collection
    del group
    gc.collect()
    
    return df, mean_features

## 🔍 Creating lag features

In [27]:
# 그룹화 기준 피처 중 '상품ID'가 포함된 파생 피처명을 담을 리스트
item_mean_features = []

# ['월ID', '상품ID']로 그룹화한 월간 평균 판매량 파생 피처 생성
all_data, item_mean_features = add_mean_features(df=all_data,
                                                 mean_features=item_mean_features,
                                                 idx_features=['월ID', '상품ID'])

# ['월ID', '상품ID', '도시']로 그룹화한 월간 평균 판매량 파생 피처 생성
all_data, item_mean_features = add_mean_features(df=all_data,
                                                 mean_features=item_mean_features,
                                                 idx_features=['월ID', '상품ID', '도시'])

In [28]:
# 그룹화 기준 피처 중 '상점ID'가 포함된 파생 피처명을 담을 리스트
shop_mean_features = []

# ['월ID', '상점ID', '상품분류ID']로 그룹화한 월간 평균 판매량 파생 피처 생성
all_data, shop_mean_features = add_mean_features(df=all_data, 
                                                 mean_features=shop_mean_features,
                                                 idx_features=['월ID', '상점ID', '상품분류ID'])

In [29]:
def add_lag_features(df, lag_features_to_clip, idx_features, 
                     lag_feature, nlags=3, clip=False):
    # 시차 피처 생성에 필요한 DataFrame 부분만 복사 
    df_temp = df[idx_features + [lag_feature]].copy() 

    # 시차 피처 생성 
    for i in range(1, nlags+1):
        # 시차 피처명 
        lag_feature_name = lag_feature +'_시차' + str(i)
        # df_temp 열 이름 설정 
        df_temp.columns = idx_features + [lag_feature_name]
        # df_temp의 date_block_num 피처에 1 더하기 
        df_temp['월ID'] += 1
        # idx_feature를 기준으로 df와 df_temp 병합하기
        df = df.merge(df_temp.drop_duplicates(), 
                      on=idx_features, 
                      how='left')
        # 결측값 0으로 대체 
        df[lag_feature_name] = df[lag_feature_name].fillna(0)
        # 0 ~ 20 사이로 제한할 시차 피처명을 lag_features_to_clip에 추가
        if clip: 
            lag_features_to_clip.append(lag_feature_name)
    
    # 데이터 다운캐스팅
    df = downcast(df, False)
    # 가비지 컬렉션
    del df_temp
    gc.collect()
    
    return df, lag_features_to_clip

In [30]:
lag_features_to_clip = [] # 0 ~ 20 사이로 제한할 시차 피처명을 담을 리스트
idx_features = ['월ID', '상점ID', '상품ID'] # 기준 피처

# idx_features를 기준으로 월간 판매량의 세 달치 시차 피처 생성
all_data, lag_features_to_clip = add_lag_features(df=all_data, 
                                                  lag_features_to_clip=lag_features_to_clip,
                                                  idx_features=idx_features,
                                                  lag_feature='월간 판매량', 
                                                  nlags=3,
                                                  clip=True)

In [31]:
all_data.head().T

,0,1,2,3,4
월ID,0.000000,0.000000,0.000000,0.000000,0.000000
상점ID,59.000000,59.000000,59.000000,59.000000,59.000000
상품ID,22154.000000,2552.000000,2554.000000,2555.000000,2564.000000
월간 판매량,1.000000,0.000000,0.000000,0.000000,0.000000
평균 판매가,999.000000,0.000000,0.000000,0.000000,0.000000
판매건수,1.000000,0.000000,0.000000,0.000000,0.000000
도시,30.000000,30.000000,30.000000,30.000000,30.000000
상품분류ID,37.000000,58.000000,58.000000,56.000000,59.000000
첫 판매월,0.000000,0.000000,0.000000,0.000000,0.000000
대분류,5.000000,7.000000,7.000000,7.000000,7.000000


In [32]:
lag_features_to_clip

['월간 판매량_시차1', '월간 판매량_시차2', '월간 판매량_시차3']

In [33]:
# idx_features를 기준으로 판매건수 피처의 세 달치 시차 피처 생성
all_data, lag_features_to_clip = add_lag_features(df=all_data, 
                                                  lag_features_to_clip=lag_features_to_clip,
                                                  idx_features=idx_features,
                                                  lag_feature='판매건수', 
                                                  nlags=3)

# idx_features를 기준으로 평균 판매가 피처의 세 달치 시차 피처 생성
all_data, lag_features_to_clip = add_lag_features(df=all_data, 
                                                  lag_features_to_clip=lag_features_to_clip,
                                                  idx_features=idx_features,
                                                  lag_feature='평균 판매가', 
                                                  nlags=3)

In [34]:
# 예상해야하는 test 데이터 기준으로 34로 월 number 지정
X_test_temp = all_data[all_data['월ID'] == 34]
X_test_temp[item_mean_features].sum()

상품ID별 평균 판매량       0.0
상품ID 도시별 평균 판매량    0.0
dtype: float32

In [35]:
# idx_features를 기준으로 item_mean_features 요소별 시차 피처 생성
for item_mean_feature in item_mean_features:
    all_data, lag_features_to_clip = add_lag_features(df=all_data, 
                                                      lag_features_to_clip=lag_features_to_clip, 
                                                      idx_features=idx_features, 
                                                      lag_feature=item_mean_feature, 
                                                      nlags=3,
                                                      clip=True)
# item_mean_features 피처 제거
all_data = all_data.drop(item_mean_features, axis=1)

In [36]:
shop_mean_features

['상점ID 상품분류ID별 평균 판매량']

In [37]:
# ['월ID', '상점ID', '상품분류ID']를 기준으로 shop_mean_features 요소별 시차 피처 생성
for shop_mean_feature in shop_mean_features:
    all_data, lag_features_to_clip = add_lag_features(df=all_data,
                                                      lag_features_to_clip=lag_features_to_clip, 
                                                      idx_features=['월ID', '상점ID', '상품분류ID'], 
                                                      lag_feature=shop_mean_feature, 
                                                      nlags=3,
                                                      clip=True)
# shop_mean_features 피처 제거
all_data = all_data.drop(shop_mean_features, axis=1)

In [38]:
# 월ID 3미만인 데이터 제거
all_data = all_data.drop(all_data[all_data['월ID'] < 3].index)

### 🔍 Generating Derived Features


In [39]:
all_data['월간 판매량 시차평균'] = all_data[['월간 판매량_시차1',
                                          '월간 판매량_시차2', 
                                          '월간 판매량_시차3']].mean(axis=1)

In [40]:
# 0 ~ 20 사이로 값 제한
all_data[lag_features_to_clip + ['월간 판매량', '월간 판매량 시차평균']] = all_data[lag_features_to_clip +['월간 판매량', '월간 판매량 시차평균']].clip(0, 20)

In [41]:
all_data['시차변화량1'] = all_data['월간 판매량_시차1']/all_data['월간 판매량_시차2']
all_data['시차변화량1'] = all_data['시차변화량1'].replace([np.inf, -np.inf], 
                                                        np.nan).fillna(0)

all_data['시차변화량2'] = all_data['월간 판매량_시차2']/all_data['월간 판매량_시차3']
all_data['시차변화량2'] = all_data['시차변화량2'].replace([np.inf, -np.inf], 
                                                        np.nan).fillna(0)

In [42]:
all_data['신상여부'] = all_data['첫 판매월'] == all_data['월ID']

In [43]:
all_data['첫 판매 후 기간'] = all_data['월ID'] - all_data['첫 판매월']

In [44]:
all_data['월'] = all_data['월ID'] % 12

In [45]:
# 첫 판매월, 평균 판매가, 판매건수 피처 제거
all_data = all_data.drop(['첫 판매월', '평균 판매가', '판매건수'], axis=1)

# 4. Training a Model and Making Predictions : `Public Score: 0.87701`

Using `all_data` to `'submission-authur1.csv'`

In [46]:
all_data = downcast(all_data, False) # 데이터 다운캐스팅

In [47]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8029765 entries, 827196 to 8856960
Data columns (total 31 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   월ID                      int8   
 1   상점ID                     int8   
 2   상품ID                     int16  
 3   월간 판매량                   int8   
 4   도시                       int8   
 5   상품분류ID                   int8   
 6   대분류                      int8   
 7   월간 판매량_시차1               int8   
 8   월간 판매량_시차2               int8   
 9   월간 판매량_시차3               int8   
 10  판매건수_시차1                 int8   
 11  판매건수_시차2                 int8   
 12  판매건수_시차3                 int8   
 13  평균 판매가_시차1               float64
 14  평균 판매가_시차2               float64
 15  평균 판매가_시차3               float64
 16  상품ID별 평균 판매량_시차1         float32
 17  상품ID별 평균 판매량_시차2         float32
 18  상품ID별 평균 판매량_시차3         float32
 19  상품ID 도시별 평균 판매량_시차1      float32
 20  상품ID 도시별 평균 판매량_시차2      float32
 21  상품I

In [48]:
# 훈련 데이터 (피처)
X_train = all_data[all_data['월ID'] < 33]
X_train = X_train.drop(['월간 판매량'], axis=1)
# 검증 데이터 (피처)
X_valid = all_data[all_data['월ID'] == 33]
X_valid = X_valid.drop(['월간 판매량'], axis=1)
# 테스트 데이터 (피처)
X_test = all_data[all_data['월ID'] == 34]
X_test = X_test.drop(['월간 판매량'], axis=1)

# 훈련 데이터 (타깃값)
y_train = all_data[all_data['월ID'] < 33]['월간 판매량']
# 검증 데이터 (타깃값)
y_valid = all_data[all_data['월ID'] == 33]['월간 판매량']

# # 가비지 컬렉션 : 뒤에서 전처리를 지속적으로 이용할 것으로 수행 X
# del all_data
# gc.collect();

In [49]:
# import lightgbm as lgb
from lightgbm import early_stopping, log_evaluation # 버전차이로 지원하지 않는 부분 설정

start_time = datetime.now()

# LightGBM 하이퍼파라미터
params = {'metric': 'rmse',  # 평가지표: RMSE
          'num_leaves': 255,
          'learning_rate': 0.005,  # 이전 학습율 0.01
          'feature_fraction': 0.75,  # 추가1
          'bagging_fraction': 0.75,  # 추가2
          'bagging_freq': 5,  # 추가3
          'force_col_wise': True,
          'random_state': 10}

# categorical 피처 설정 : 도시, 대분류, 월 추가
cat_features = ['상점ID', '도시', '상품분류ID', '대분류', '월']

# LightGBM 훈련 및 검증 데이터셋
dtrain = lgb.Dataset(X_train, y_train)
dvalid = lgb.Dataset(X_valid, y_valid)


# LightGBM 모델 훈련

# 100회 반복마다 검증지표 출력(verbose_eval 파라미터 대체)
# early_stopping_rounds=150 대체 추가
callbacks = [log_evaluation(period=100),  # verbose_eval 대체
            early_stopping(stopping_rounds=150)]  # early_stopping_rounds 대체

lgb_model = lgb.train(params=params,
                      train_set=dtrain,
                      num_boost_round=1500,
                      valid_sets=(dtrain, dvalid),
                      categorical_feature=cat_features,
                      callbacks=callbacks) # verbose_eval, early_stopping_rounds 대체

end_time = datetime.now()
print(f"runtime: {end_time - start_time}")

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Total Bins 3657
[LightGBM] [Info] Number of data points in the train set: 7593847, number of used features: 30
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 0.307756
Training until validation scores don't improve for 150 rounds
[100]	training's rmse: 1.0178	valid_1's rmse: 0.913728
[200]	training's rmse: 0.912629	valid_1's rmse: 0.841626
[300]	training's rmse: 0.859658	valid_1's rmse: 0.812341
[400]	training's rmse: 0.830685	valid_1's rmse: 0.800648
[500]	training's rmse: 0.811176	valid_1's rmse: 0.796129
[600]	training's rmse: 0.797613	valid_1's rmse: 0.794879
[700]	training's rmse: 0.78732	valid_1's rmse: 0.794375
[800]	training's rmse: 0.77898	valid_1's rmse: 0.794011
[900]	training's rmse: 0.771955	valid_1's rmse: 0.794313
[1000]	training's rmse: 0.765998	valid_1's rmse: 0.794295
Early stopping, best iteratio

In [50]:
# 예측
preds = lgb_model.predict(X_test).clip(0, 20)

# 제출 파일 생성
submission['item_cnt_month'] = preds
submission.to_csv('submission-authur.csv', index=False)

In [51]:
del X_train, y_train, X_valid, y_valid, X_test, lgb_model, dtrain, dvalid
gc.collect();

valid_1's rmse: 0.79395

Public Score: 0.87701


# 5. Using the Hyperparameters Derived from _Bayesian Optimization_ 

## 5-1. in previous notebook : `Public Score: 0.88809`


최적화된 하이퍼파라미터: {'bagging_fraction': 0.7949387309032754, 'bagging_freq': 9, 'feature_fraction': 0.5, 'learning_rate': 0.01, 'num_leaves': 213}

`'submission-private-5-1.csv'`

In [52]:
# 훈련 데이터 (피처)
X_train = all_data[all_data['월ID'] < 33]
X_train = X_train.drop(['월간 판매량'], axis=1)
# 검증 데이터 (피처)
X_valid = all_data[all_data['월ID'] == 33]
X_valid = X_valid.drop(['월간 판매량'], axis=1)
# 테스트 데이터 (피처)
X_test = all_data[all_data['월ID'] == 34]
X_test = X_test.drop(['월간 판매량'], axis=1)

# 훈련 데이터 (타깃값)
y_train = all_data[all_data['월ID'] < 33]['월간 판매량']
# 검증 데이터 (타깃값)
y_valid = all_data[all_data['월ID'] == 33]['월간 판매량']

# 가비지 컬렉션
# del all_data
# gc.collect();

In [53]:
# import lightgbm as lgb
# from lightgbm import early_stopping, log_evaluation # 버전차이로 지원하지 않는 부분 설정

start_time = datetime.now()

# 얻은 LightGBM 하이퍼파라미터
params = {'metric': 'rmse',  # 평가지표: RMSE
          'bagging_fraction': 0.7949387309032754, 
          'bagging_freq': 9, 
          'feature_fraction': 0.5, 
          'learning_rate': 0.01, 
          'num_leaves': 213,
          'force_col_wise': True,
          'random_state': 10}

# categorical 피처 설정 : 도시, 대분류, 월 추가
cat_features = ['상점ID', '도시', '상품분류ID', '대분류', '월']

# LightGBM 훈련 및 검증 데이터셋
dtrain = lgb.Dataset(X_train, y_train)
dvalid = lgb.Dataset(X_valid, y_valid)


# LightGBM 모델 훈련

# 100회 반복마다 검증지표 출력(verbose_eval 파라미터 대체)
# early_stopping_rounds=150 대체 추가
callbacks = [log_evaluation(period=100),  # verbose_eval 대체
            early_stopping(stopping_rounds=150)]  # early_stopping_rounds 대체

lgb_model = lgb.train(params=params,
                      train_set=dtrain,
                      num_boost_round=1500,
                      valid_sets=(dtrain, dvalid),
                      categorical_feature=cat_features,
                      callbacks=callbacks)

end_time = datetime.now()
print(f"runtime: {end_time - start_time}")

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Total Bins 3657
[LightGBM] [Info] Number of data points in the train set: 7593847, number of used features: 30
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 0.307756
Training until validation scores don't improve for 150 rounds
[100]	training's rmse: 0.929383	valid_1's rmse: 0.848691
[200]	training's rmse: 0.848294	valid_1's rmse: 0.802654
[300]	training's rmse: 0.817299	valid_1's rmse: 0.794773
[400]	training's rmse: 0.799351	valid_1's rmse: 0.792995
[500]	training's rmse: 0.786615	valid_1's rmse: 0.792804
[600]	training's rmse: 0.776371	valid_1's rmse: 0.793823
Early stopping, best iteration is:
[479]	training's rmse: 0.789316	valid_1's rmse: 0.792472
runtime: 0:07:28.152741


In [54]:
# 예측
preds = lgb_model.predict(X_test).clip(0, 20)

# 제출 파일 생성
submission['item_cnt_month'] = preds
submission.to_csv('submission-private-5-1.csv', index=False)

valid_1's rmse: 0.792472

public score: 0.88809


## 5-2. with [실전문제 preprocessing] : `Public Score: 0.89295`



최적화된 파라미터: {'bagging_fraction': 0.50446175516352, 'bagging_freq': 8, 'feature_fraction': 0.50446175516352, 'learning_rate': 0.005538531127282626, 'num_leaves': 135}


In [55]:
# 훈련 데이터 (피처)
X_train = all_data[all_data['월ID'] < 33]
X_train = X_train.drop(['월간 판매량'], axis=1)
# 검증 데이터 (피처)
X_valid = all_data[all_data['월ID'] == 33]
X_valid = X_valid.drop(['월간 판매량'], axis=1)
# 테스트 데이터 (피처)
X_test = all_data[all_data['월ID'] == 34]
X_test = X_test.drop(['월간 판매량'], axis=1)

# 훈련 데이터 (타깃값)
y_train = all_data[all_data['월ID'] < 33]['월간 판매량']
# 검증 데이터 (타깃값)
y_valid = all_data[all_data['월ID'] == 33]['월간 판매량']

# 가비지 컬렉션
# del all_data
# gc.collect();

In [56]:
# import lightgbm as lgb
# from lightgbm import early_stopping, log_evaluation # 버전차이로 지원하지 않는 부분 설정

start_time = datetime.now()

# 얻은 LightGBM 하이퍼파라미터
params = {'metric': 'rmse',  # 평가지표: RMSE
          'bagging_fraction': 0.50446175516352, 
          'bagging_freq': 8, 
          'feature_fraction': 0.50446175516352, 
          'learning_rate': 0.005538531127282626, 
          'num_leaves': 135,
          'force_col_wise': True,
          'random_state': 10}

# categorical 피처 설정 : 도시, 대분류, 월 추가
cat_features = ['상점ID', '도시', '상품분류ID', '대분류', '월']

# LightGBM 훈련 및 검증 데이터셋
dtrain = lgb.Dataset(X_train, y_train)
dvalid = lgb.Dataset(X_valid, y_valid)


# LightGBM 모델 훈련

callbacks = [log_evaluation(period=100),  # verbose_eval 대체
            early_stopping(stopping_rounds=150)]  # early_stopping_rounds 대체

lgb_model = lgb.train(params=params,
                      train_set=dtrain,
                      num_boost_round=1500,
                      valid_sets=(dtrain, dvalid),
                      categorical_feature=cat_features,
                      callbacks=callbacks)

end_time = datetime.now()
print(f"runtime: {end_time - start_time}")

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Total Bins 3657
[LightGBM] [Info] Number of data points in the train set: 7593847, number of used features: 30
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 0.307756
Training until validation scores don't improve for 150 rounds
[100]	training's rmse: 1.02178	valid_1's rmse: 0.911698
[200]	training's rmse: 0.924857	valid_1's rmse: 0.839315
[300]	training's rmse: 0.879219	valid_1's rmse: 0.811362
[400]	training's rmse: 0.855026	valid_1's rmse: 0.799988
[500]	training's rmse: 0.839214	valid_1's rmse: 0.795199
[600]	training's rmse: 0.828281	valid_1's rmse: 0.792876
[700]	training's rmse: 0.819763	valid_1's rmse: 0.791665
[800]	training's rmse: 0.813004	valid_1's rmse: 0.791262
[900]	training's rmse: 0.807528	valid_1's rmse: 0.791318
Early stopping, best iteration is:
[827]	training's rmse: 0.811659	valid_1's rmse: 0

In [57]:
# 예측
preds = lgb_model.predict(X_test).clip(0, 20)

# 제출 파일 생성
submission['item_cnt_month'] = preds
submission.to_csv('submission-private-5-2.csv', index=False)

valid_1's rmse: 0.791056

public score: 0.89295

# 6. Trying Additional Feature Engineering 

## 6-1. 근접한 달에 labeling - 1 : `Public Score: 0.87945`

(EDA에서 확인한 바) 예측해야하는 11월은 대부분 9, 10월과 비슷한 수준의 판매량을 보였으므로, 해당하는 월들에 대하여 labeling 진행  

In [58]:
Dec = [11, 23, 35]
nearNov = [val for i in Dec for val in [i-1, i-2, i-3]]
nearNov

[10, 9, 8, 22, 21, 20, 34, 33, 32]

In [59]:
all_data['nearNov'] = all_data['월ID'].isin(nearNov).astype(int)
all_data.sample(5)

,월ID,상점ID,상품ID,월간 판매량,도시,상품분류ID,대분류,월간 판매량_시차1,월간 판매량_시차2,월간 판매량_시차3,...,상점ID 상품분류ID별 평균 판매량_시차1,상점ID 상품분류ID별 평균 판매량_시차2,상점ID 상품분류ID별 평균 판매량_시차3,월간 판매량 시차평균,시차변화량1,시차변화량2,신상여부,첫 판매 후 기간,월,nearNov
6438944,24,16,3360,0,10,30,3,0,0,0,...,1.115790,1.069149,1.321429,0.000000,0.0,0.0,0,24,0,0
7383920,28,16,4577,0,10,56,7,0,0,0,...,0.093750,0.088235,0.032258,0.000000,0.0,0.0,0,28,4,0
6562593,24,38,13549,0,17,69,8,0,0,0,...,0.387097,0.254237,0.206897,0.000000,0.0,0.0,0,8,0,0
2784833,9,15,14841,10,9,65,8,1,5,1,...,0.340909,1.022222,0.351852,2.333333,0.2,5.0,0,5,9,1
4520355,16,5,21836,0,3,37,5,0,0,0,...,0.094508,0.082310,0.073110,0.000000,0.0,0.0,0,1,4,0


In [60]:
# 훈련 데이터 (피처)
X_train = all_data[all_data['월ID'] < 33]
X_train = X_train.drop(['월간 판매량'], axis=1)
# 검증 데이터 (피처)
X_valid = all_data[all_data['월ID'] == 33]
X_valid = X_valid.drop(['월간 판매량'], axis=1)
# 테스트 데이터 (피처)
X_test = all_data[all_data['월ID'] == 34]
X_test = X_test.drop(['월간 판매량'], axis=1)

# 훈련 데이터 (타깃값)
y_train = all_data[all_data['월ID'] < 33]['월간 판매량']
# 검증 데이터 (타깃값)
y_valid = all_data[all_data['월ID'] == 33]['월간 판매량']

# 가비지 컬렉션
# del all_data
# gc.collect();

In [61]:
# import lightgbm as lgb
# from lightgbm import early_stopping, log_evaluation # 버전차이로 지원하지 않는 부분 설정

start_time = datetime.now()

# LightGBM 하이퍼파라미터
params = {'metric': 'rmse',  # 평가지표: RMSE
          'num_leaves': 255,
          'learning_rate': 0.005,  # 이전 학습율 0.01
          'feature_fraction': 0.75,  # 추가1
          'bagging_fraction': 0.75,  # 추가2
          'bagging_freq': 5,  # 추가3
          'force_col_wise': True,
          'random_state': 10}

# categorical 피처 설정 : 도시, 대분류, 월 추가
cat_features = ['상점ID', '도시', '상품분류ID', '대분류', '월']

# LightGBM 훈련 및 검증 데이터셋
dtrain = lgb.Dataset(X_train, y_train)
dvalid = lgb.Dataset(X_valid, y_valid)


# LightGBM 모델 훈련

# 100회 반복마다 검증지표 출력(verbose_eval 파라미터 대체)
# early_stopping_rounds=150 대체 추가
callbacks = [log_evaluation(period=100),  # verbose_eval 대체
            early_stopping(stopping_rounds=150)]  # early_stopping_rounds 대체

lgb_model = lgb.train(params=params,
                      train_set=dtrain,
                      num_boost_round=1500,
                      valid_sets=(dtrain, dvalid),
                      categorical_feature=cat_features,
                      callbacks=callbacks)

end_time = datetime.now()
print(f"runtime: {end_time - start_time}")

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Total Bins 3659
[LightGBM] [Info] Number of data points in the train set: 7593847, number of used features: 31
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 0.307756
Training until validation scores don't improve for 150 rounds
[100]	training's rmse: 1.01856	valid_1's rmse: 0.91328
[200]	training's rmse: 0.91327	valid_1's rmse: 0.842114
[300]	training's rmse: 0.86017	valid_1's rmse: 0.812148
[400]	training's rmse: 0.830187	valid_1's rmse: 0.801014
[500]	training's rmse: 0.811293	valid_1's rmse: 0.79731
[600]	training's rmse: 0.798036	valid_1's rmse: 0.795851
[700]	training's rmse: 0.787631	valid_1's rmse: 0.79527
[800]	training's rmse: 0.779159	valid_1's rmse: 0.795479
Early stopping, best iteration is:
[699]	training's rmse: 0.78773	valid_1's rmse: 0.795252
runtime: 0:11:18.094885


In [62]:
# 예측
preds = lgb_model.predict(X_test).clip(0, 20)

# 제출 파일 생성
submission['item_cnt_month'] = preds
submission.to_csv('submission-private-6-1.csv', index=False)

valid_1's rmse: 0.795252

Public Score: 0. 0.87945

### 2. 근접한 달에 labeling - 2 : `Public Score: 0.88068`

nearNov에 0, 0.5, November에 1로 차등적으로 값 주는 방식으로 수정하여 시도

In [63]:
Dec = [11, 23, 35]
Nov = [val-1 for val in Dec]
nearNov = [val for i in Dec for val in [i-2, i-3]]

all_data['nearNov'] = np.where(all_data['월ID'].isin(nearNov), 0.5, 0)
all_data['nearNov'] = np.where(all_data['월ID'].isin(Nov), 1, 0)
all_data.sample(5)

,월ID,상점ID,상품ID,월간 판매량,도시,상품분류ID,대분류,월간 판매량_시차1,월간 판매량_시차2,월간 판매량_시차3,...,상점ID 상품분류ID별 평균 판매량_시차1,상점ID 상품분류ID별 평균 판매량_시차2,상점ID 상품분류ID별 평균 판매량_시차3,월간 판매량 시차평균,시차변화량1,시차변화량2,신상여부,첫 판매 후 기간,월,nearNov
1078737,3,44,21508,0,20,66,8,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1,0,3,0
3198513,11,19,2415,0,12,30,3,1,2,0,...,1.888476,2.032389,2.298851,1.000000,0.5,0.0,0,11,11,0
8303768,32,52,10600,0,25,43,6,0,0,0,...,0.057471,0.052632,0.074766,0.000000,0.0,0.0,0,1,8,0
2085867,7,56,6599,1,28,25,3,1,0,0,...,0.500000,0.305556,0.297297,0.333333,0.0,0.0,0,7,7,0
1658820,5,50,8807,0,24,40,5,0,0,1,...,0.291667,0.301327,0.339123,0.333333,0.0,0.0,0,5,5,0


In [64]:
# 훈련 데이터 (피처)
X_train = all_data[all_data['월ID'] < 33]
X_train = X_train.drop(['월간 판매량'], axis=1)
# 검증 데이터 (피처)
X_valid = all_data[all_data['월ID'] == 33]
X_valid = X_valid.drop(['월간 판매량'], axis=1)
# 테스트 데이터 (피처)
X_test = all_data[all_data['월ID'] == 34]
X_test = X_test.drop(['월간 판매량'], axis=1)

# 훈련 데이터 (타깃값)
y_train = all_data[all_data['월ID'] < 33]['월간 판매량']
# 검증 데이터 (타깃값)
y_valid = all_data[all_data['월ID'] == 33]['월간 판매량']

# 가비지 컬렉션
# del all_data
# gc.collect();

In [65]:
# import lightgbm as lgb
# from lightgbm import early_stopping, log_evaluation # 버전차이로 지원하지 않는 부분 설정

start_time = datetime.now()

# LightGBM 하이퍼파라미터
params = {'metric': 'rmse',  # 평가지표: RMSE
          'num_leaves': 255,
          'learning_rate': 0.005,  # 이전 학습율 0.01
          'feature_fraction': 0.75,  # 추가1
          'bagging_fraction': 0.75,  # 추가2
          'bagging_freq': 5,  # 추가3
          'force_col_wise': True,
          'random_state': 10}

# categorical 피처 설정 : 도시, 대분류, 월 추가
cat_features = ['상점ID', '도시', '상품분류ID', '대분류', '월']

# LightGBM 훈련 및 검증 데이터셋
dtrain = lgb.Dataset(X_train, y_train)
dvalid = lgb.Dataset(X_valid, y_valid)


# LightGBM 모델 훈련

# 100회 반복마다 검증지표 출력(verbose_eval 파라미터 대체)
# early_stopping_rounds=150 대체 추가
callbacks = [log_evaluation(period=100),  # verbose_eval 대체
            early_stopping(stopping_rounds=150)]  # early_stopping_rounds 대체

lgb_model = lgb.train(params=params,
                      train_set=dtrain,
                      num_boost_round=1500,
                      valid_sets=(dtrain, dvalid),
                      categorical_feature=cat_features,
                      callbacks=callbacks) # verbose_eval, early_stopping_rounds 대체

end_time = datetime.now()
print(f"runtime: {end_time - start_time}")

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Total Bins 3659
[LightGBM] [Info] Number of data points in the train set: 7593847, number of used features: 31
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 0.307756
Training until validation scores don't improve for 150 rounds
[100]	training's rmse: 1.01866	valid_1's rmse: 0.913874
[200]	training's rmse: 0.913423	valid_1's rmse: 0.84161
[300]	training's rmse: 0.860369	valid_1's rmse: 0.811409
[400]	training's rmse: 0.830438	valid_1's rmse: 0.799544
[500]	training's rmse: 0.81143	valid_1's rmse: 0.795141
[600]	training's rmse: 0.798102	valid_1's rmse: 0.793469
[700]	training's rmse: 0.787683	valid_1's rmse: 0.792922
[800]	training's rmse: 0.779295	valid_1's rmse: 0.793229
Early stopping, best iteration is:
[716]	training's rmse: 0.786275	valid_1's rmse: 0.792896
runtime: 0:11:34.163772


In [66]:
# 예측
preds = lgb_model.predict(X_test).clip(0, 20)

# 제출 파일 생성
submission['item_cnt_month'] = preds
submission.to_csv('submission-private-6-2.csv', index=False)

valid_1's rmse: 0.792896

public score : 0.88068

